In [ ]:
#   Ensure imports are in the virtual environment
#   !pip install pandas scikit-learn matplotlib seaborn

In [ ]:
#   Imports
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

In [ ]:
housing_dataframe = pd.read_csv("housing.csv")
housing_dataframe.head()

In [ ]:
#   Check dataframe info
housing_dataframe.info()

In [ ]:
#   Check classes in ocean_proximity
housing_dataframe["ocean_proximity"].value_counts()

In [ ]:
#   Describe data
housing_dataframe.describe()

In [ ]:
housing_dataframe.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
#   Create train and test set manually(Only for today)
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
naive_train_set, naive_test_set = split_train_test(housing_dataframe, 0.2)
print(len(naive_train_set), len(naive_test_set))

In [ ]:
#   Problem: Function will generate different examples every time we run it
#            thus we'll see the whole dataset in the   long run
#   Solution: Compute hash of each instance's   identifier and put that instance
#             in the test set if the hash is lower or equal to 20% of the maximum hash value
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
#   adds an index column
housing_with_id = housing_dataframe.reset_index()
withhash_train_set, withhash_test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
#   Using longitude and latitude as indices
housing_with_id["id"] = housing_dataframe["longitude"] * 1000 + housing_dataframe["latitude"]
ll_train_set, ll_test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [ ]:
#   Using sklearn
train_set, test_set = train_test_split(housing_dataframe, test_size=0.2, random_state=42)

In [ ]:
#   There stands the possibility that the data may be
#   stratified thus leading to some misrepresentation
#   in some of the strata in the test set
#   Example: Median income. To ensure complete
#   representation in the test set, we can create an
#   income category attribute to represent various
#   strata. Each strata must have sufficient data to
#   ensure that the stratum's importance is not biased
housing_dataframe["income_cat"] = pd.cut(housing_dataframe["median_income"],bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

In [ ]:
#   Plot histogram
housing_dataframe.income_cat.hist()

In [ ]:
#   Stratified sampling based on income_cat
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing_dataframe, housing_dataframe["income_cat"]):
    strat_train_set = housing_dataframe.loc[train_index]
    strat_test_set = housing_dataframe.loc[test_index]

In [ ]:
#   Check income category proportions in the test set
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
#   Check same in train set
strat_train_set["income_cat"].value_counts() / len(strat_train_set)

In [ ]:
#   Note: Doing the same with the other split options
#   will show that they are quite skewed whereas the
#   stratified sampling is almost identical
#   Remove income_cat to restore data to its original
#   state
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
#   Visualizing Geographical Data
#   alpha option to 0.1 makes it easier to visualize
#   the places where there is a high density of data
#   points
housing_dataframe.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
#   s = district's population
#   c = color represents the price
#   cmap: jet(ranges from blue to red)
housing_dataframe.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=housing_dataframe["population"]/100, label="population", figsize=(10, 7), c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()
#   Plot shows that housing prices are very much
#   related to the location and population density

In [ ]:
#   Looking for correlations
#   Compute standard correlation coefficient
#   (Pearson's r)
#   range 1(+ve correlation) to -1 (-ve correlation)
correlation_matrix = housing_dataframe.corr()
correlation_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
#   Using pandas' scatter matrix
#   Plots numerical attributes against every other
#   numerical attributes
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing_dataframe[attributes], figsize=(12, 8))

In [ ]:
#   median_income seems like a promising attribute
housing_dataframe.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

In [ ]:
#   Experimenting with attribute combinations
housing_dataframe["rooms_per_household"] = housing_dataframe["total_rooms"] / housing_dataframe["households"]
housing_dataframe["bedrooms_per_room"] = housing_dataframe["total_bedrooms"] / housing_dataframe["total_rooms"]
housing_dataframe["population_per_household"] = housing_dataframe["population"] / housing_dataframe["households"]

In [ ]:
#   Get correlation matrix of all
corr_matrix = housing_dataframe.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)
#   We notice that bedrooms per room is more correlated
#   than total number of bedrooms. This exploration
#   step may help us gain more insights for our
#   exploration step

In [ ]:
#   Separate label/target from dataset
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
#   Data cleaning
#   Working with missing values

#   Option 1: Drop rows with missing values
housing.dropna(subset=["total_bedrooms"])

#   Option 2: Drop columns with the missing values
housing.drop("total_bedrooms", axis=1)

#   Option 3: Fill missing values
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)

In [ ]:
#   Fill missing values using SimpleImputer
imputer = SimpleImputer(strategy="median")
#   Median can only be calculated on numerical
#   attributes. Drop categorical attributes
#   and create copy of the data
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
#   Use trained imputer to transorm training set by
#   replacing missing values with the median
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [ ]:
#   Handling text and Categorical Attributes
cat_attribs = [ col for col in housing_dataframe.columns if housing_dataframe[col].dtype == "object" ]
housing_cat = housing_dataframe[cat_attribs]
housing_cat.head(10)

In [ ]:
#   Encode text
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

In [ ]:
#   Problem: Algorithm may assume that nearer
#   attributes are more similar than two distant values
#   e.g <1H OCEAN(0), INLAND(1) and NEAR OCEAN(4)
#   Solution: One hot encoding
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()
housing_cat_1hot = onehot_encoder.fit_transform(housing_cat)
housing_cat_1hot
#   Sparse matrix stores location of the non zero elements

In [ ]:
# Custom transformer
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]

        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
housing_extra_attribs.shape

In [ ]:
#   Feature Scaling
#   min-max scaling/normalization: MinMaxScaler range 0-1
#   standardization: (i-mean)/standard_deviation. Less prone to outliers
#   Use pipeline for transformations

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_attribs = list(housing_num)

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler()),
])

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
#   Select and train a model
#   Using a LinearRegression model
from sklearn.linear_model import LinearRegression

linear_regression_model = LinearRegression()
linear_regression_model.fit(housing_prepared, housing_labels)

In [ ]:
#   Testing on a few instances on the training dataset
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
linear_regression_model.predict(some_data_prepared)

In [ ]:
#   compare with labels
list(some_labels)

In [ ]:
#   Calculate error
#   lre = LinearRegressor
from sklearn.metrics import mean_squared_error
housing_predictions_lre = linear_regression_model.predict(housing_prepared)
linear_model_mse = mean_squared_error(housing_labels, housing_predictions_lre)
linear_model_rmse = np.sqrt(linear_model_mse)
linear_model_rmse

In [ ]:
#   Model is underfitting.
#   Why?
#   1.  Not enough features
#   2.  Model is not powerful enough
#   Solutions
#   1.  Choose a more powerful model
#   2.  Train with better features
#   3.  Reduce constraints on model(however it's not regularized so this option is ruled out)
from sklearn.tree import DecisionTreeRegressor

decision_tree_regressor = DecisionTreeRegressor()
decision_tree_regressor.fit(housing_prepared, housing_labels)

In [ ]:
#   dtr = DecisionTreeRegressor
housing_predictions_dtr = decision_tree_regressor.predict(housing_prepared)
decision_tree_regressor_mse = mean_squared_error(housing_labels, housing_predictions_dtr)
decision_tree_regressor_rmse = np.sqrt(decision_tree_regressor_mse)
decision_tree_regressor_rmse
#   We are not sure if model overfitted or not
#   Test set will be touched iff we are sure our model is ready

In [ ]:
#   Cross Validation
#   To evaluate model, options are:
#       1.  Split training set into train and val
#       2.  Using K-fold cross-validation feature
#   k-fold randomly splits data into k folds then it trains and evaluates model k times,
#   picking a differnet fold for evaluation every time and training on the other folds k-1 times
from sklearn.model_selection import cross_val_score
drt_scores = cross_val_score(decision_tree_regressor, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
dtr_rmse_scores = np.sqrt(-drt_scores)

In [ ]:
#   Function to display cross_val_score results
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
display_scores(dtr_rmse_scores)

In [ ]:
#   Compare with linear regression model
lre_scores = cross_val_score(linear_regression_model, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lre_rmse_scores = np.sqrt(-lre_scores)
display_scores(lre_rmse_scores)
#   We learn that the decision tree is overfitting horribly

In [ ]:
#   K = 10, scoring=negative mse
def train_with_cross_val(model, X, y):
    scores = cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=10)
    rmse_scores = np.sqrt(-scores)
    return display_scores(rmse_scores)

#   Calculate mse
def rmse(model, X, y):
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)
    return np.sqrt(mse)

In [ ]:
#   Try with random forests
from sklearn.ensemble import RandomForestRegressor
random_forest_regressor = RandomForestRegressor()
random_forest_regressor.fit(housing_prepared, housing_labels)
rmse(random_forest_regressor, housing_prepared, housing_labels)

In [ ]:
train_with_cross_val(random_forest_regressor, housing_prepared, housing_labels)

In [ ]:
#   Fine tune model
#   Grid search - evaluates all possible combinations of hyperparameter values using cross validation
#   param_grid the grid search will explore 12 + 6 = 18 combinations of RandomForestRegressor
#   hyperparameter values, and it will train each model five times (since we are
#   using five-fold cross validation)
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'n_estimators':[3,10,30],
        'max_features':[2,4,6,8]
    },
    {
        'bootstrap':[False],
        'n_estimators':[3,10],
        'max_features':[2,3,4]
    }
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
#   get evaluation scores
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
#   Grid search is great while using a small search space for hyperparameters
#   For large search space use RandomizedSearchCV
#   Ensemble Methods combine the methods that perform the best e.g Random Forests vs Decision Trees
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["rooms_per_hhold", "population_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
#   Evaluation on the test set
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))